### Week 5 Day 4

AutoGen Core - Distributed

I'm only going to give a Teaser of this!!

Partly because I'm unsure how relevant it is to you. If you'd like me to add more content for this, please do let me know..

In [1]:
from dataclasses import dataclass
from autogen_core import AgentId, MessageContext, RoutedAgent, message_handler
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.tools.langchain import LangChainToolAdapter
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain.agents import Tool
from IPython.display import display, Markdown

from dotenv import load_dotenv

load_dotenv(override=True)

ALL_IN_ONE_WORKER = False

### Start with our Message class

In [2]:

@dataclass
class Message:
    content: str

### And now - a host for our distributed runtime

In [3]:
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntimeHost

host = GrpcWorkerAgentRuntimeHost(address="localhost:50052")
host.start() 

### Let's reintroduce a tool

In [4]:
serper = GoogleSerperAPIWrapper()
langchain_serper =Tool(name="internet_search", func=serper.run, description="Useful for when you need to search the internet")
autogen_serper = LangChainToolAdapter(langchain_serper)

In [5]:
instruction1 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons in favor of choosing AutoGen; the pros of AutoGen."

instruction2 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons against choosing AutoGen; the cons of Autogen."

judge = "You must make a decision on whether to use AutoGen for a project. \
Your research team has come up with the following reasons for and against. \
Based purely on the research from your team, please respond with your decision and brief rationale."

### And make some Agents

In [6]:
class Player1Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client, tools=[autogen_serper], reflect_on_tool_use=True)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    
class Player2Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client, tools=[autogen_serper], reflect_on_tool_use=True)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    
class Judge(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client)
        
    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        message1 = Message(content=instruction1)
        message2 = Message(content=instruction2)
        inner_1 = AgentId("player1", "default")
        inner_2 = AgentId("player2", "default")
        response1 = await self.send_message(message1, inner_1)
        response2 = await self.send_message(message2, inner_2)
        result = f"## Pros of AutoGen:\n{response1.content}\n\n## Cons of AutoGen:\n{response2.content}\n\n"
        judgement = f"{judge}\n{result}Respond with your decision and brief explanation"
        message = TextMessage(content=judgement, source="user")
        response = await self._delegate.on_messages([message], ctx.cancellation_token)
        return Message(content=result + "\n\n## Decision:\n\n" + response.chat_message.content)


In [7]:
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntime

if ALL_IN_ONE_WORKER:

    worker = GrpcWorkerAgentRuntime(host_address="localhost:50052")
    await worker.start()

    await Player1Agent.register(worker, "player1", lambda: Player1Agent("player1"))
    await Player2Agent.register(worker, "player2", lambda: Player2Agent("player2"))
    await Judge.register(worker, "judge", lambda: Judge("judge"))

    agent_id = AgentId("judge", "default")

else:

    worker1 = GrpcWorkerAgentRuntime(host_address="localhost:50052")
    await worker1.start()
    await Player1Agent.register(worker1, "player1", lambda: Player1Agent("player1"))

    worker2 = GrpcWorkerAgentRuntime(host_address="localhost:50052")
    await worker2.start()
    await Player2Agent.register(worker2, "player2", lambda: Player2Agent("player2"))

    worker = GrpcWorkerAgentRuntime(host_address="localhost:50052")
    await worker.start()
    await Judge.register(worker, "judge", lambda: Judge("judge"))
    agent_id = AgentId("judge", "default")




In [8]:
response = await worker.send_message(Message(content="Go!"), agent_id)

In [9]:
display(Markdown(response.content))

## Pros of AutoGen:
Here are several advantages of using AutoGen in your AI Agent project:

1. **Multi-Agent Collaboration**: AutoGen excels in creating multi-agent systems that allow various AI agents to interact and collaborate to solve complex problems, enabling more sophisticated solutions.

2. **Customizability**: The framework offers flexibility and customization options, allowing developers to tailor agents to specific needs and tasks, which can significantly enhance the functionality and effectiveness of AI applications.

3. **Complex Problem Solving**: AutoGen is particularly beneficial for tackling complex tasks that may be too challenging for a single AI model. This capability allows for negotiating and managing intricate workflows seamlessly.

4. **Efficiency and Speed**: It helps streamline the development and orchestration of AI workflows, leading to quicker completion of tasks and a reduction in time spent on manual processes.

5. **Production-Ready Maturity**: AutoGen is designed to be robust and reliable, making it suitable for production environments without extensive modification.

6. **Cross-Language Support**: The platform supports multiple programming languages, providing additional flexibility for developers to integrate it into diverse tech stacks.

Overall, AutoGen can significantly enhance the efficiency and capabilities of an AI Agent project through its multi-agent approach, customization options, and support for complex workflows.

TERMINATE

## Cons of AutoGen:
Here are some reasons against choosing AutoGen for your new AI Agent project:

1. **Resource-Intensive**: AutoGen can be computationally expensive due to its requirement for continuous interactions with large models, which may increase overhead costs.

2. **Complexity**: The framework presents considerable complexity that can pose a challenge, especially for newcomers. This steep learning curve may slow down development and implementation.

3. **Limited Customization**: AutoGen may offer less flexibility in custom integrations compared to other frameworks, potentially restricting the extent to which you can tailor the solution to your needs.

4. **Reliability Concerns**: While suitable for research and prototyping, AutoGen may fall short in reliability and consistency for production-level, customer-facing applications.

5. **High Costs for Multi-Agent Workflows**: Utilizing multi-agent workflows can be expensive, making it difficult to justify the investment for real-world applications.

6. **Limited AI Capabilities**: It may not be the best fit for complex AI-driven automation tasks, potentially limiting creative problem-solving possibilities in your project.

These drawbacks should be carefully considered in the context of your specific project requirements. 

TERMINATE



## Decision:

Based on the evaluation of the pros and cons of using AutoGen for our AI Agent project, I recommend proceeding with the use of AutoGen. The advantages of multi-agent collaboration, customizability, and the ability to efficiently solve complex problems outweigh the concerns related to resource intensity and complexity. Given that our project demands a robust framework capable of managing intricate workflows with reliable outcomes, AutoGen aligns well with our objectives despite its higher costs and potential learning curve.

TERMINATE

In [10]:
await worker.stop()
if not ALL_IN_ONE_WORKER:
    await worker1.stop()
    await worker2.stop()

In [11]:
await host.stop()